In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

In [11]:
df = pd.read_csv('C:/Users/omira/Desktop/Anticipate-Energy-Needs/buildings_cleaned.csv')

In [10]:
df.isnull().sum()

BuildingType           0
PrimaryPropertyType    0
YearBuilt              0
NumberofFloors         0
PropertyGFATotal       0
NumberofBuildings      0
SiteEUI(kBtu/sf)       0
SiteEnergyUse(kBtu)    0
Electricity(kWh)       0
NaturalGas(therms)     0
TotalGHGEmissions      0
ENERGYSTARScore        0
dtype: int64

In [3]:
# 2. Modelling

# Expliquer pourquoi on choisi ces variables
df_subset = df[['ENERGYSTARScore', 'YearBuilt', 'NumberofFloors', 'BuildingType', 'Electricity(kWh)']]
df_subset = df_subset.dropna()

# Target

y = df_subset['Electricity(kWh)']
X = df_subset.drop(columns=['Electricity(kWh)'])

In [4]:
# II. Transformations des variables - Feature Engineering

# a. Dummyfication (encoding des variables categorielles)
X = pd.get_dummies(X)

# b. transformation log
# Faire avec le prix
X['ENERGYSTARScore'] = np.log(X['ENERGYSTARScore'])

# c. Transformation métier / business
# X['habitant_per_meter'] = X['number_people'] / X['meter_square']

In [5]:
# Train - Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
# Entrainez sur les données train uniquement

# 1. Premier Modèle (linear)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Linear Regression; Regression Tree, Gradient Boosting, etc...

In [7]:
#  Evaluation - MSE
MSE = mean_squared_error(y_test, y_pred)

In [8]:
MSE

2205132501135.444

In [9]:
# 2. Test with RegressionTree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

y_pred_tree = clf.predict(X_test)
MSE_tree = mean_squared_error(y_test, y_pred_tree)

MSE_tree > MSE
# Regression modele is better

ValueError: Unknown label type: 'continuous'

In [24]:
# 3. GridSeachCV -> Utilise pour trouver les meilleurs hyperpramètres du modèle
model_linear = LinearRegression()
parameters = {'fit_intercept':[True, False], 'normalize':[True, False]}
grid_linear = GridSearchCV(model_linear, parameters, cv=3)

grid_linear.fit(X_train, y_train)

sorted(grid_linear.cv_results_.keys())

best_model = grid_linear.best_estimator_

y_pred_lr_best_model = best_model.predict(X_test)
MSE_linear_regression_best_model = mean_squared_error(y_test, y_pred_lr_best_model)

MSE_linear_regression_best_model > MSE

C:\Users\omira\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
C:\Users\omira\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model =

True

In [28]:
# TODO: Faire pour les autres modèles (decisionTree)
# 4. Feature Importance
# Explication

best_model.coef_

# Plotter les coefficients avec les valeurs
# Regarder si certains sont nuls (tests)

# Only for linear regression
# y_elec = f(X)
# y_elec = alpha1 * ENERGYSTARSCORE + alpha2 * YearBuilt + alpha_i * ...

# DecisionTree (explication)

# tree.plot_tree(clf, max_depth=3)
# plt.show()
# text_representation = tree.export_text(clf, max_depth=3)
# print(text_representation)

array([-1.86732039e+05,  7.11604696e+03,  2.09648989e+05, -9.35733746e+06,
       -1.54640020e+07, -1.36621471e+07, -1.40600964e+07, -1.28017122e+07,
       -1.27846881e+07,  0.00000000e+00, -1.30920848e+07])